# Einsendeaufgabe 1: Numerische Genauigkeit und Gleichungssysteme (100 Punkte)
In dieser Aufgabe sollen Sie ein wenig mehr Erfahrung mit NumPy und numerischen Methoden gewinnen.  
Zur Erinnerung empfehle ich an dieser Stelle, die Definition der [IEEE-Fließkommazahlen](https://de.wikipedia.org/wiki/IEEE_754) zu wiederholen.  

## Addition von Zahlen (20 Punkte) 

Gegeben sei ein Array *array*, dass 100 mal die Zahl $10^{-16}$ enthält und einmal (als ersten Eintrag) die Zahl $1$. 

In [2]:
import numpy as np

array = np.concatenate(([1], np.full(100, 1e-16)))
print(array)

ModuleNotFoundError: No module named 'numpy'

**Aufgabe:** Addieren Sie alle Werte in `array`:
- mit der Funktion `np.sum()`
- mit einer Schleife, die von **vorne nach hinten** über `array` iteriert
- mit einer Schleife, die von **hinten nach vorne** über `array` iteriert

_Points:_ 6

In [ ]:
sum_numpy = np.sum(array)
print(f"{sum_numpy=}")

sum_forward = 0
for i in array:
    sum_forward += i
print(f"{sum_forward=}")

sum_reverse = 0
for j in array[::-1]:
    sum_reverse += j
print(f"{sum_reverse=}")

<!-- BEGIN QUESTION -->

**Aufgabe:** Erklären Sie die Ergebnisse. Wie werden die Zahlen 1 und 1e-17 im Computer dargestellt? (ausführliche Erläuterung erforderlich!)  

_Points:_ 12

Die np.sum() Funktion in Python wird verwendet, um alle Elemente in einem Array zu addieren. Da Python in der Regel mit einer höheren Genauigkeit als die Gleitkommazahlen im Computer rechnet, sollte der Wert des Ergebnisses nahe bei 1 liegen. Allerdings könnte es aufgrund von Rundungsfehlern zu kleinen Abweichungen kommen.

Wenn eine Schleife von vorne nach hinten über das Array iteriert und die Werte addiert, könnte es aufgrund von Rundungsfehlern zu einem nicht exakten Ergebnis kommen. Die Rundungsfehler können auftreten, wenn die sehr kleine Zahl 10^-16 viele Male addiert wird, da der Computer nur eine begrenzte Anzahl von Bits zur Darstellung von Gleitkommazahlen hat.

Wenn eine Schleife von hinten nach vorne über das Array iteriert und die Werte addiert, kann dies zu einem genaueren Ergebnis führen, da die kleinen Zahlen zuerst addiert werden und die Rundungsfehler möglicherweise kleiner sind.

Die Zahlen 1 und 10^-16 werden im Computer im binären Format gespeichert. Die Zahl 1 wird im binären Format als "1" dargestellt, während die Zahl 10^-16 im binären Format als eine Reihe von Nullen mit einer Eins am Ende dargestellt wird. Da der Computer eine begrenzte Anzahl von Bits zur Darstellung von Gleitkommazahlen hat, kann es zu Rundungsfehlern kommen, wenn sehr kleine Zahlen viele Male addiert werden.

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Aufgabe:** Welches Verfahren verwendet `np.sum()` für die Addition?

_Points:_ 2

np.sum() verwendet das Verfahren der elementweisen Addition, bei dem die Elemente eines Arrays oder einer Liste zusammengezählt werden. Das bedeutet, dass jeder Wert in einem Array mit jedem anderen Wert addiert wird, um das Endergebnis zu erhalten.

<!-- END QUESTION -->

## Grenzwerte von Funktionen (20 Punkte)

Geben sei die Funktion $f(x)=\frac{e^x-1}{x}$

Es gilt: $lim_{x \to 0}f(x) = 1$

**Aufgabe:** Schreiben Sie eine Python Funktion `f(x)`, mit deren Hilfe Sie $f(x)$ für $x= 10^{-1}, 10^{-2}, \cdots , 10^{-15}$ mit NumPy berechnen und geben Sie das Ergebnis aus.

_Points:_ 8

In [ ]:
import numpy as np


def f(x): return (np.exp(x)-1)


input_numbers = [10**-1, 10**-2, 10**-3, 10**-4, 10**-5, 10**-6, 10**-7, 10**-8, 10**-9, 10**-10, 10**-11, 10**-12, 10**-13, 10**-14, 10**-15]
result_limits = []
for x in range(len(input_numbers)):
    result_limits.append(f(x))

print(f"{result_limits=}")

In [ ]:
input_numbers_2 = [10**-16, 10**-17, 10**-18, 10**-19, 10**-20]
result_limits_2 = []
for x in range(len(input_numbers_2)):
    result_limits_2.append(f(x))
print(f"{result_limits_2=}")

<!-- BEGIN QUESTION -->

**Aufgabe:** Was fällt auf? Welche Erklärung haben Sie für das Ergebnis? (Ausführliche Erklärung erforderlich) 

_Points:_ 12

_Type your answer here, replacing this text._

<!-- END QUESTION -->

## Gauß-Verfahren und LU-Zerlegung (60 Punkte) 
Obwohl man, wie wir sehen werden, lineare Gleichungssysteme einfach mit SciPy lösen kann, wollen wir in dieser Aufgabe  einmal selbst eine sogenannte LU-Zerlegung bzw. das Gauß-Verfahren implementieren. Außerdem werden wir diesen Code auch später noch verwenden, wenn wir uns mit Performance-Optimierung und Parallelisierung beschäftigen.

Das Gauß-Verfahren ist ein wichtiges Verfahren zum Lösen von linearen Gleichungssystemen, wir gehen hier schrittweise mit der Implementierung vor.

**Aufgabe:** Implementieren Sie die Funktion `b_solve` zur Lösung einer Dreiecksgleichung der Form:

$\left[ {\begin{array}{cccc} 
a_{11} & a_{12} & \cdots & a_{1n}\\ 
0 & a_{22} & \cdots & a_{2n}\\ 
\vdots & \vdots & \ddots & \vdots \\ 
0 & 0 & \cdots & a_{nn}\\ 
\end{array} } \right] \left[{\begin{array}{c}  
x_1\\ 
x_2\\ 
\vdots\\ 
x_n\\ 
\end{array}}\right] =  
\left[{\begin{array}{c}  
b_1\\ 
b_2\\ 
\vdots\\ 
b_n\\ 
\end{array}}\right]$ 

Hier beginnt man mit der Löung der letzten Zeile, denn es gilt:

$x_n = b_n/a_{nn} $
und dann

$x_{n-1} =  (b_{n-1}-a_{(n-1),n}*x_{n})/a_{(n-1),(n-1)}$

$x_{n-2} =  (b_{n-2}-(a_{(n-2),n}*x_{n}+a_{(n-2),(n-1)}*x_{n-1})/a_{(n-2),(n-2)}$


*Hinweis*: Achten Sie darauf, dass ihr Code nicht nur korrekt, sondern auch effizient ist. 

_Points:_ 12

In [ ]:
import numpy as np


def b_solve(A, b):
    """Solve a system of linear equations A * x = b where A is an upper triangular matrix."""
    n = len(b)
    x = np.zeros(n)

    for i in range(n-1, -1, -1):
        if A[i, i] == 0:
            raise ValueError("Matrix ist singulär!")
        sum_ax = 0
        for j in range(i+1, n):
            sum_ax += A[i, j] * x[j]
        x[i] = (b[i] - sum_ax) / A[i, i]

    return x

**Aufgabe:** Implementieren Sie die LU-Zerlegung einer Matrix mit Pivotisierung.  
Die Funktion `lu_decomposition` soll die Matrizen L und U berechnen.  
Zu Beginn wird der Funktion eine Matrix A übergeben.  


Implementieren Sie die Funktion "in-place", d.h. die Ergebnisse für L und U werden direkt in A geschrieben.  
Dabei wird die ursprüngliche Matrix überschrieben.  
Die Funktion gibt den Pivot-Vektor zurück, der die folgende Form haben sollte:

$ 
P=\left[{\begin{array}{ccc} 
0 & 1& 0\\ 
0 & 0& 1\\ 
1 & 0& 0\\ 
\end{array} } \right] \Rightarrow \left[{\begin{array}{c}  
1\\ 
2\\ 
0\\ 
\end{array}}\right] 
$ 

D.h. es wird nur der Index des Wertes in jeder Zeile gespeichert, der nicht Null ist.

Wenn man dies macht, kann man später für die Lösung einfach den Vektor `bc = b[P]` verwenden, um auch $b$ richtig zu pivotisieren.  

Hinweise: 
- Die Pivot-Zeile lässt sich am einfachsten mit `np.argmax` finden. Die Zeile mit dem höchsten Betrag können Sie mit `np.abs` finden.

- Tauschen zweier Elemente in einem NumPy Array: `v[[a, b]] = v[[b, a]]`

- Versuchen Sie, die interne Vektorisierung von NumPy im Eliminationsschritt zu nutzen. Verwenden Sie dazu die Slicing-Syntax, um die innerste Schleife zu ersetzen.

_Points:_ 15

In [ ]:
def lu_decomposition(A):
    """
    Implement LU-Decomposition of a square matrix.
    This function works in-place i.e. it stores information about L and U in A and returns the pivot vector.
    """
    n = A.shape[0]
    p = np.arrange(n)

    for i in range(n):
        max_row = i + np.argmax(np.abs(A[i:n, 1]))
        if i != max_row:
            A[[i, max_row], :] = A[[max_row, i], :]
            p[[i, max_row]] = p[[max_row, i]]
    
    for j in range(i + 1, n):
        A[j, i] /= A[i, i]
        A[j, i+1] -= A[j, i] * A[i, i+1:]
    return p

<!-- BEGIN QUESTION -->

**Aufgabe:** Schreiben Sie eine Funktion `custom_solve`, welche mit Hilfe von `lu_decomposition` ein lineares Gleichungssystem der Form $A*x=b$ löst.  

Hinweis: Versuchen Sie, die interne Vektorisierung von NumPy zu nutzen. Verwenden Sie dazu die Slicing-Syntax, um die innerste Schleife zu ersetzen.

_Points:_ 12

In [ ]:
def custom_solve(A, b):
    """Solve a system of linear equations A * x = b and return result vector x."""
    A = A.copy()
    p = lu_decomposition(A)

    b = b[p]

    n = A.shape[0]
    x = np.zeros_like(b)
    y = np.zeros_like(b)

    for i in range(n):
        y[i] = b[i] - np.dot(A[i, :i], y[:i])
    
    for i in range(n-1, -1, -1):
        x[i] = (y[i] - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]
    
    return x

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Aufgabe:** Schreiben Sie eine Testfunktion, die überprüft, ob die Lösung ihrer Implementierung des Gauß-Verfahrens richtig ist.  
Testen Sie Gleichungssysteme der Größe $16 \times 16$, $32\times 32$ und $64 \times 64$.

Sie können zum Vergleich die SciPy Funktion `linalg.solve` verwenden:

_Points:_ 6

In [ ]:
# Beispiel linalg
import scipy.linalg as linalg

n = 128
rng = np.random.default_rng(seed=123)
A = rng.random((n, n))
b = rng.random(n)
c = linalg.solve(A, b)

In [ ]:
import scipy.linalg as linalg

# test different dimensions
def test():
    sizes = [16, 32, 64]
    rng = np.random.default_rng(seed=123)

    for size in sizes:
        A = rng.random((size, size))
        b = rng.random(size)

        x_custom = custom_solve(A,b)
        x_scipy = linalg.solve(A, b)

        diff = np.linalg.norm(x_custom - x_scipy)
        print(f"Größe: {size} x {size} - Differenz: {diff}")

test()

<!-- END QUESTION -->

**Aufgabe:** Wir wollen nun weiter die *fertigen* SciPy Funktionen betrachten. Mit den Funktionen `lu` und `lu_factor` aus dem Packet `scipy.linalg `  können Sie eine LU-Dekomposition durchführen [[1]](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lu_factor.html#scipy.linalg.lu_factor) und [[2]](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lu.html).  
Beide Funktionen haben unter anderem den Parameter `overwrite_a`, der `False` oder `True` sein kann. 

Evaluieren Sie die Performance (Laufzeit) der LU-Zerlegung einer 4096x4096 Matrix, wenn Sie jeweils die Funktionen `lu_factor` und `lu` mit `overwrite_a = True` oder `overwrite_a = False` verwenden.

In der Liste `minimums` sollen die minimalen Laufzeiten in folgender Reihenfolge stehen:
1. `lu_factor` mit `overwrite_a = False`
2. `lu_factor` mit `overwrite_a = True`
3. `lu` mit `overwrite_a = False`
4. `lu` mit `overwrite_a = True`

Hinweis: Da die Matrix A bei der Verwendung von `overwrite_a = True` verändert wird, kann es zu einer Vereinfachung der Berechnung kommen, wenn die Funktion mehrfach nacheinander ausgeführt wird. Aus diesem Grund muss die `timeit` Funktion [repeat](https://docs.python.org/3/library/timeit.html#timeit.repeat) mit `repeat = 2` und `number = 1`  verwendet werden. Die Matrix wird im `setup` Parameter erstellt (aus Zufallswerten mit dem vorgegebenen "random number generator").

_Points:_ 6

In [ ]:
from scipy.linalg import lu_factor, lu
from timeit import repeat

n = 8192
number = 1
repetitions = 2
# store all four minimal runtimes in this list
minimums = []
rng = np.random.default_rng(seed=123)

...

**Aufgabe:** Untersuchen Sie den Speicherverbrauch der 4 Varianten mit memray.

Um den Einfluss der anderen Berechnungen zu minimieren, empfehlen wir hier, die Berechnungen in eine neue Datei zu schreiben und mit `!` in einer Code-Zelle memray's Command-Line Interface zu verwenden. 

Am besten tracen Sie nur die LU Funktion mit einem Context Manager!
Verwenden Sie jeweils folgende Dateinamen als `file_name` Parameter in `memray.Tracker`:

- lu_inplace.bin
- lu_notinplace.bin
- lu_factor_inplace.bin
- lu_factor_notinplace.bin

Beantworten Sie bitte die folgenden Fragen:
* Welche Version verwendet am meisten Speicher? 
* Bei welcher Version ist der Speicherverbrauch am konstantesten? 

_Points:_ 9

In [ ]:
%%file lu_inplace.py
import memray
import numpy as np
from scipy.linalg import lu

n = 8192
rng = np.random.default_rng(seed=123)
...

In [ ]:
%%file lu_notinplace.py
import memray
import numpy as np
from scipy.linalg import lu

n = 8192
rng = np.random.default_rng(seed=123)
...

In [ ]:
%%file lu_factor_inplace.py
import memray
import numpy as np
from scipy.linalg import lu_factor

n = 8192
rng = np.random.default_rng(seed=123)
...

In [ ]:
%%file lu_factor_notinplace.py
import memray
import numpy as np
from scipy.linalg import lu_factor

n = 8192
rng = np.random.default_rng(seed=123)
...

In [ ]:
...

In [ ]:
#!rm -rf lu_* memray-*